In [1]:
import gym
import numpy as np
from Actor import Actor
import torch
from Loader import Loader
import torch.utils.data.dataloader as dataloader

In [17]:
env = gym.make("LunarLanderContinuous-v2")
state_size = np.prod(list(env.observation_space.shape))
action_size = np.prod(list(env.action_space.shape))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [18]:
ob = env.reset()
a = ob
# print(ob)
# print(ob[None])

ob = ob[None]
ob = np.append (ob, ob, axis = 0)
ob = np.append (ob, ob, axis = 0)
print(ob)

[[ 3.27396393e-04  9.47147757e-01  3.31492245e-02  4.35616843e-01
  -3.72622249e-04 -7.50884041e-03  0.00000000e+00  0.00000000e+00]
 [ 3.27396393e-04  9.47147757e-01  3.31492245e-02  4.35616843e-01
  -3.72622249e-04 -7.50884041e-03  0.00000000e+00  0.00000000e+00]
 [ 3.27396393e-04  9.47147757e-01  3.31492245e-02  4.35616843e-01
  -3.72622249e-04 -7.50884041e-03  0.00000000e+00  0.00000000e+00]
 [ 3.27396393e-04  9.47147757e-01  3.31492245e-02  4.35616843e-01
  -3.72622249e-04 -7.50884041e-03  0.00000000e+00  0.00000000e+00]]


In [19]:
ob = torch.from_numpy(ob).float()

In [20]:
def weights_init(m):
    if hasattr(m, 'weight'):
        torch.nn.init.xavier_uniform_(m.weight)

In [21]:
agent = Actor(n_ip=3, n_op=action_size).apply(weights_init)

In [22]:
if next(agent.parameters()).is_cuda:
    print("CUDA")
else:
    print("CPU")

CUDA


In [23]:
datasett = [ ((1,2,3,4),(1,2)), ((5,6,7,8),(5,6)), ((7,8,9,0),(7,8)) ]

In [24]:
expert_loader = Loader(data_collected=datasett)
loader = dataloader.DataLoader(expert_loader, batch_size=2, shuffle=True)

In [25]:
for x,y in loader:
    print(x)
    break

tensor([[7., 8., 9., 0.],
        [1., 2., 3., 4.]], device='cuda:0')


In [26]:
print(x[0])

tensor([7., 8., 9., 0.], device='cuda:0')


In [27]:
b = [[1,2,3], [4,5,6]]
# b = [[1,2,3]]
a = torch.Tensor(b).to("cuda")
print(a)


tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')


In [28]:
with torch.no_grad():
    mean, std = agent(a)
    m = mean.cpu().numpy()
    s = std.cpu().numpy()
    print("m\n", m)
    print("\n\ns\n", s)

m
 [[ 0.39692912 -0.55686307]
 [ 0.68864477 -0.7101314 ]]


s
 [[1.1054512e+00 1.0000000e-03]
 [1.5086987e+00 1.0000000e-03]]


In [29]:
m = [[1,2],[10,20], [100, 200]]
s = [[1,1],[1,1], [1,1]]

In [30]:

m = torch.tensor(m).float()
s = torch.tensor(s).float()

sampled_action = torch.normal(mean=m, std=s)
print(sampled_action)

tensor([[ 1.6259,  2.0199],
        [10.7599, 20.0397]])


In [84]:
m = [[1,2],[10,20], [100, 200]]
s = [[1,1],[1,1], [1,1]]

chnk = len(m[0])
print(chnk)

m1 = torch.tensor(m).flatten().float()
s1 = torch.tensor(s).flatten().float()

print(m1)

print(s1)
sampled_action = torch.normal(mean=m1, std=s1).reshape(-1, chnk).numpy()
#sampled_action.flatten()
print(sampled_action.flatten())


2
tensor([  1.,   2.,  10.,  20., 100., 200.])
tensor([1., 1., 1., 1., 1., 1.])
[  0.2805382   1.445059    9.795682   20.995651   99.35505   199.93478  ]


In [85]:
# fix log prob
def get_log_prob(mean, standard_deviation, actions):
    m = mean
    s = standard_deviation
    log_prob = torch.distributions.Normal(loc=m, scale=s).log_prob(actions).sum(-1)
    return  log_prob

In [87]:
actions = torch.tensor(sampled_action).flatten()
log_probs = get_log_prob(mean=m1, standard_deviation=s1, actions=m1)
print(log_probs)
print(torch.exp(log_probs))

tensor(-5.5136)
tensor(0.0040)


In [3]:
import numpy as np

In [17]:
a = [1,2,3]
b = [10,20,30]
a = tuple(a)
b = tuple(b)

d = {}
d[(a,b)] = 99

a = [11,12,13]
b = [110,120,130]
a = tuple(a)
b = tuple(b)
d[(a,b)] = 299

print(d)

d = list(d.items())

print(d)

for i in d:    
    key, value = i
    state = list(key[0])
    action = list(key[1])
    print()
    print("state : ", state)
    print("action : ", action)
    print("value : ", value)

{((1, 2, 3), (10, 20, 30)): 99, ((11, 12, 13), (110, 120, 130)): 299}
[(((1, 2, 3), (10, 20, 30)), 99), (((11, 12, 13), (110, 120, 130)), 299)]

state :  [1, 2, 3]
action :  [10, 20, 30]
value :  99

state :  [11, 12, 13]
action :  [110, 120, 130]
value :  299
